In [ ]:

# clear_output()
### LOGIN

f = open(".env.json", "r")
env = json.load(f)
f.close()
scrapper.login(env["username_animeflv"], env["password_animeflv"])
del env["password_animeflv"]

In [48]:
global job, file_job
job_name = "1"

folder_job = os.path.join("jobs",job_name)
file_job = os.path.join(folder_job,"info.json")

if not os.path.isdir(folder_job):
    
    # sys.exit(0)
else:
    print("That job doesn't exist")
    

def close_job(is_completed, df):
    global scrapper
    global file_job, job 
    print("Stopping scraper...", is_completed, type(df))
    scrapper.stop = True
    
    print("Saving job...")
    
    pending = []
    if not is_completed:
        index_seen = scrapper.cache.get("index_seen",-1)
        pending = scrapper.cache.get("urls",[])
        if len(pending) > 0:
            pending = pending[index_seen+1:]
          
        df = scrapper.cache.get("df", None)
        
    job['urls_queued'] = pending
    
    job['urls_error'] = scrapper.cache.get("urls_error", [])
    save_job(file_job, job)
    
    if df is not None:
        df.to_csv(f"temp/db_{job['name']}_items{job['execution_time']}.csv", index=False)
    
    print(f"Closing with: {len(scrapper.cache.get('urls',[])) -len(job['urls_queued'])} completed, {len(job['urls_queued'])} pending urls and {len(job['urls_error'])} error urls")
    

def signal_handler(sig, frame):
    close_job(False, None)
    
    try:
        sys.exit()
    except:
        print("Problems with sys.exit!")
        sys.exit()
        
signal.signal(signal.SIGINT, signal_handler)

df = pd.DataFrame()

print(f"Begin work, don't open or touch files in {folder_job}")

timers = {
    "timer_load_page": 2,
    "timer_load_disqus": 2,
    "timer_load_reactions": 1
}

state = job["state"]

if state != State.END:
    
    f = open(".env.json", "r")
    env = json.load(f)
    f.close()
    scrapper = ScrapperAnimeFLV(headless=False)  # "/home/magody/chromedriver_linux64/chromedriver"
    scrapper.driver.get("https://www3.animeflv.net/")
    scrapper.wait(5)
    # clear_output()
    ### LOGIN
    scrapper.login(env["username_animeflv"], env["password_animeflv"])
    del env["password_animeflv"]


if state == State.BEGIN:
    print("Begining")
    df = scrapper.scrape_items(timers, url_begin="https://www3.animeflv.net/browse?page=1")
    close_job(True, df)
elif state == State.PENDING_ITEMS:
    scrapper.stop = False
    urls_items = job["urls_queued"]
    urls_items.extend(job["urls_error"])  # try again the errors
    job["urls_error"] = []
    job["execution_time"] += 1
    print(f"Pending work...{len(urls_items)} urls in queue")
    df, completed = scrapper.scrape_items(timers, urls=urls_items)
    print("scrape_items ended")
    if completed and len(scrapper.cache["urls_error"]) == 0: 
        job["state"] = State.END
    close_job(completed, df)
elif state == State.END:
    print("Work already completed!")
else:
    print("Can't handle this state")
    
    

Exception ignored in: <function ScrapperAnimeFLV.__del__ at 0x7f8d02c3f040>
Traceback (most recent call last):
  File "/tmp/ipykernel_35982/4006993225.py", line 364, in __del__
  File "/tmp/ipykernel_35982/4006993225.py", line 359, in close
  File "/home/magody/anaconda3/lib/python3.8/site-packages/selenium/webdriver/remote/webdriver.py", line 937, in close
    self.execute(Command.CLOSE)
  File "/home/magody/anaconda3/lib/python3.8/site-packages/selenium/webdriver/remote/webdriver.py", line 424, in execute
    self.error_handler.check_response(response)
  File "/home/magody/anaconda3/lib/python3.8/site-packages/selenium/webdriver/remote/errorhandler.py", line 247, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.InvalidSessionIdException: Message: invalid session id
Stacktrace:
#0 0x55cc5e322533 (/home/magody/.local/share/undetected_chromedriver/chromedriver (deleted)+0xffffffffffaeb532)
#1 0x55cc5e08107f (/home/magody/.local/share/un

DESTRUCTING OBJECT
CLOSING OBJECT...


  0%|          | 0/100 [00:00<?, ?it/s]

Logged in!
Pending work...100 urls in queue


 70%|███████   | 70/100 [07:50<03:18,  6.60s/it]

Discuss error 1. 

 87%|████████▋ | 87/100 [09:51<01:28,  6.83s/it]

Discuss error 1. 

100%|██████████| 100/100 [11:21<00:00,  6.81s/it]

Finished!
scrape_items ended
Stopping scraper...Completed: True DF type: <class 'pandas.core.frame.DataFrame'>
Saving job...
Closing with: 100 completed, 0 pending urls and 0 error urls


CLOSING OBJECT...
